Script to test the python mzml loader 
==

In [1]:
import sys
# sys.path.append('/Users/simon/git/lda/code/')
sys.path.append('/Users/joewandy/git/lda/code/')

In [2]:
%load_ext autoreload
%autoreload 2

Load data using pymzml

In [3]:
# data_dir = '/Users/simon/Dropbox/Bioresearch/Meta_clustering/MS2LDA/pymzml/comparisons/'
data_dir = '/Users/joewandy/Dropbox/Meta_clustering/MS2LDA/pymzml/comparisons/'

In [4]:
from ms2lda_feature_extraction import LoadMZML
input_set = [data_dir + 'Beer_3_T10_POS.mzML']
peaklist = data_dir + 'Beer_3_full1.csv'
loader = LoadMZML(peaklist = peaklist,isolation_window = 0.5, mz_tol = 5, rt_tol = 10.0)
ms1,ms2,metadata = loader.load_spectra(input_set)

Loading spectra from /Users/joewandy/Dropbox/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_T10_POS.mzML
Found 6680 ms2 spectra, and 191043 individual ms2 objects
Loaded 7863 ms1 peaks from /Users/joewandy/Dropbox/Meta_clustering/MS2LDA/pymzml/comparisons/Beer_3_full1.csv
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
Peaklist filtering results in 1582 documents


No. of MS1 peaks that can be matched to the peaklist

In [5]:
len(ms1)

1582

No. of MS2 peaks

In [6]:
len(ms2)

55785

### Filtering

*SR: not sure this step is necesary?*

Try to filter the pymzml results using the same filter criteria as in method 3's

In [7]:
min_ms1_rt = 3*60  # seconds
max_ms1_rt = 21*60 # seconds
min_ms2_intensity = 5000

# filter MS1 and MS2 by retention time
ms1 = filter(lambda x: x.rt>min_ms1_rt and x.rt<max_ms1_rt, ms1)
ms2 = filter(lambda x: x[3].rt>min_ms1_rt and x[3].rt<max_ms1_rt, ms2)

# filter the MS2 peaks by intensity
ms2 = filter(lambda x: x[2]>min_ms2_intensity, ms2)

In [8]:
len(ms1), len(ms2)

(1459, 22665)

In [32]:
print ms1[0]
print ms2[0]

71.0727281885_605.939025879
(53.002601623535156, 564.7979736328125, 5252.98583984375, <ms2lda_feature_extraction.MS1 object at 0x10e72b1d0>, 'Beer_3_T10_POS.mzML', 96994.0)


Now load method 3's results

In [9]:
import pandas as pd
from IPython.display import display, HTML

In [10]:
ms1_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms1.csv', 
                      index_col=0)
ms2_df = pd.read_csv(data_dir + 'Beer3_POS_method3_csv_full1_ms2.csv', 
                      index_col=0)

In [11]:
ms1_df.shape[0], ms2_df.shape[0]

(1289, 24287)

### Try to match the MS1 peaks from both results

Justin: *what would be of interest is to select the more abundant ones (>1E6) and see how the figures are*

In [12]:
min_ms1_intensity = 1E6

In [19]:
from matching import MatchFeature, SimpleMatching
from collections import defaultdict

prepare pymzml results

In [51]:
# assign each MS2 to its parent
ms2_peaks = defaultdict(list)
for peak in ms2:
    parent = peak[3]
    ms2_peaks[parent].append(peak)

pymzml_res = []
for peak in ms1:
    if peak.intensity > min_ms1_intensity:
        metadata = {
            'origin': 'pyzml',
            'ms2_masses': sorted([x[0] for x in ms2_peaks[peak]])
        }
        f = MatchFeature(peak.mz, peak.rt, peak.intensity, metadata)
        pymzml_res.append(f)

prepare method_3 results

In [52]:
# assign each MS2 to its parent
ms2_peaks = defaultdict(list)
for idx, row in ms2_df.iterrows():
    parent = row['MSnParentPeakID']
    ms2_peaks[parent].append(row)    

method_3_res = []
for idx, row in ms1_df.iterrows():
    peak_id = row['peakID']
    mz = row['mz']
    rt = row['rt']
    intensity = row['intensity'] 
    if intensity > min_ms1_intensity:
        metadata = {
            'origin': 'method_3',
            'ms2_masses': sorted([x['mz'] for x in ms2_peaks[peak_id]])
        }
        f = MatchFeature(mz, rt, intensity, metadata)
        method_3_res.append(f)

This does a matching (based on the m/z and RT only) for the MS1 peaks from both results

In [53]:
m = SimpleMatching()
matches = m.run(pymzml_res, method_3_res, 2, 5)

Computing scores
Matching
747 matched feature pairs
171 unmatched features from feature_list_1
123 unmatched features from feature_list_2


In [74]:
def to_str(f):
    ms2_masses = f.metadata['ms2_masses']
    output = "mz=%f, rt=%f, intensity=%f from=%s, ms2_counts=%d\n" % (f.mz, f.rt, f.intensity, 
                                                                    f.metadata['origin'], len(ms2_masses))
    output += '\n'.join(format(x, '.4f') for x in ms2_masses)
    return output

In [75]:
for match in matches:
    items = list(match)
    if len(items) > 1: # a pair
        print to_str(items[0]), '\n', to_str(items[1])
    else:
        print to_str(items[0]) # single thing
    print

mz=86.096347, rt=276.096008, intensity=1282799.125000 from=pyzml, ms2_counts=6
53.0026
56.0497
57.0578
69.0702
86.0601
86.0964 
mz=86.096347, rt=276.096008, intensity=1282799.125000 from=method_3, ms2_counts=6
53.0026
56.0498
57.0578
69.0701
86.0601
86.0964

mz=99.044050, rt=587.219971, intensity=1300673.500000 from=pyzml, ms2_counts=10
53.0027
53.0389
55.0182
56.9651
69.0336
71.0493
82.0290
98.9848
99.0446
99.0556 
mz=99.044050, rt=587.219971, intensity=1300673.500000 from=method_3, ms2_counts=10
53.0027
53.0389
55.0182
56.9651
69.0336
71.0493
82.0290
98.9848
99.0446
99.0556

mz=157.080102, rt=622.317017, intensity=7317130.000000 from=pyzml, ms2_counts=24
56.0498
56.9651
57.0534
68.0498
69.0529
81.0445
82.0480
82.0523
83.0496
83.0561
83.0604
84.0573
84.0639
93.0452
94.0414
94.0482
95.0601
96.0575
96.0637
110.0719
111.0676
111.0743
134.0129
157.0814 
mz=157.080102, rt=622.317017, intensity=7317130.000000 from=method_3, ms2_counts=24
56.0498
56.9651
57.0534
68.0498
69.0529
81.0445
82.04